In [52]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean,udf,regexp_replace
from pyspark.sql.types import StringType
from unidecode import unidecode

In [53]:
def quitar_acentos_y_L(cadena):
    cadena_sin_acentos = unidecode(cadena)
    return cadena_sin_acentos.replace("L'", "")

In [54]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

Es vol ajuntar per districtes les variables de lloguer per mes, lloguer per m^2 i import de renda. Les variables de lloguer es troben juntes en renda i per poder separar-les s'ha de filtar pel tipus euro/mes per una banda i euro/m2 per l'altre. Per les 3 bases s'ha de fer la mitjana per any.

In [55]:
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price) AS Year_Price_month FROM rent_price WHERE "Average _Rent" = 'average rent (euro/month)' GROUP BY Year ,District''') \
  .load()
DF1.show()

+----+-------------------+------------------+
|Year|           District|  Year_Price_month|
+----+-------------------+------------------+
|2014|        Sant Andreu| 538.1388000000002|
|2015|Sarria-Sant Gervasi|1083.3508333333332|
|2016|Sarria-Sant Gervasi|1185.8641666666665|
|2017|Sarria-Sant Gervasi|1281.4125000000001|
|2018|     Horta-Guinardo|         750.91775|
|2019|           Eixample|1080.7333333333333|
|2020|          Les Corts|1263.5749999999998|
|2021|          Les Corts|1214.0666666666666|
|2022|     Sants-Montjuic| 885.6333333333333|
|2022|         Nou Barris| 675.9549999999997|
|2022|         Sant Marti|            970.36|
|2014|           Eixample| 772.4354166666667|
|2015|       Ciutat Vella| 682.9724999999999|
|2015|             Gracia|           690.794|
|2016|       Ciutat Vella|           752.025|
|2016|             Gracia| 756.1959999999999|
|2017|       Ciutat Vella|          846.1575|
|2017|             Gracia| 834.3240000000002|
|2018|          Les Corts|1238.325

In [56]:
DF2 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", '''SELECT Year, District, AVG(Price) AS Year_Price_m2 FROM rent_price WHERE "Average _Rent" = 'average rent per surface (euro/m2)' GROUP BY Year ,District''') \
  .load()
DF2.show()

+----+-------------------+------------------+
|Year|           District|     Year_Price_m2|
+----+-------------------+------------------+
|2014|Sarria-Sant Gervasi|11.842500000000001|
|2015|        Sant Andreu| 8.835555555555556|
|2016|        Sant Andreu| 9.481153846153846|
|2017|        Sant Andreu|10.764999999999999|
|2019|       Ciutat Vella|15.293750000000001|
|2019|             Gracia|13.929999999999996|
|2020|     Sants-Montjuic|13.267857142857144|
|2020|         Nou Barris|11.259999999999998|
|2020|         Sant Marti|14.040000000000001|
|2021|     Sants-Montjuic|12.678571428571427|
|2021|         Nou Barris|10.922499999999998|
|2021|         Sant Marti|13.440000000000001|
|2022|          Les Corts|14.449999999999998|
|2015|     Horta-Guinardo| 9.773170731707317|
|2016|     Horta-Guinardo|          10.67025|
|2017|     Horta-Guinardo|          11.83175|
|2018|Sarria-Sant Gervasi|15.192083333333331|
|2019|     Sants-Montjuic|12.972413793103447|
|2019|         Nou Barris|11.33749

In [57]:
DF3 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT \"Any\", Nom_Districte, AVG(Import_Euros) AS District_Income FROM renda GROUP BY \"Any\",Nom_Districte") \
  .load()
DF3.show()

+----+-------------------+------------------+
| Any|      Nom_Districte|   District_Income|
+----+-------------------+------------------+
|2021|         Nou Barris| 31362.30769230769|
|2020|       Ciutat Vella|           27431.5|
|2019|     Sants-Montjuïc| 35258.96581196581|
|2018|        Sant Andreu| 35419.98947368421|
|2018|         Sant Martí| 36379.22448979592|
|2017|        Sant Andreu| 34028.38947368421|
|2017|         Sant Martí|35001.163265306124|
|2016|        Sant Andreu| 33561.27368421053|
|2016|         Sant Martí|34506.945578231294|
|2015|     Horta-Guinardó|32367.845528455284|
|2021|          Les Corts|54134.350877192985|
|2021|             Gràcia| 42881.68965517241|
|2019|Sarrià-Sant Gervasi| 68802.73469387754|
|2018|     Horta-Guinardó| 34626.95121951219|
|2017|     Horta-Guinardó| 33374.52032520325|
|2016|     Horta-Guinardó| 32968.96747967479|
|2015|        Sant Andreu| 32916.29473684211|
|2015|         Sant Martí|33719.755102040814|
|2021|Sarrià-Sant Gervasi|        

In [58]:
quitar_acentos_y_L_udf = udf(quitar_acentos_y_L, StringType())

# Aplicar la función UDF a la columna 'texto_con_acentos' y crear una nueva columna
DF3 = DF3.withColumn('Nom_Districte', quitar_acentos_y_L_udf(DF3['Nom_Districte']))

# Mostrar el DataFrame resultante
DF3.show()

+----+-------------------+------------------+
| Any|      Nom_Districte|   District_Income|
+----+-------------------+------------------+
|2021|          Les Corts|54134.350877192985|
|2021|             Gracia| 42881.68965517241|
|2019|Sarria-Sant Gervasi| 68802.73469387754|
|2018|     Horta-Guinardo| 34626.95121951219|
|2017|     Horta-Guinardo| 33374.52032520325|
|2016|     Horta-Guinardo| 32968.96747967479|
|2015|        Sant Andreu| 32916.29473684211|
|2015|         Sant Marti|33719.755102040814|
|2021|       Ciutat Vella|28643.444444444445|
|2020|         Nou Barris|           30406.0|
|2019|     Horta-Guinardo| 35955.48780487805|
|2018|Sarria-Sant Gervasi|  67783.3469387755|
|2017|Sarria-Sant Gervasi| 63511.84693877551|
|2016|Sarria-Sant Gervasi|  61588.9693877551|
|2015|     Sants-Montjuic|31367.478632478633|
|2021|         Nou Barris| 31362.30769230769|
|2020|       Ciutat Vella|           27431.5|
|2019|     Sants-Montjuic| 35258.96581196581|
|2018|        Sant Andreu| 35419.9

In [59]:
DF3 = DF3.withColumnRenamed("Nom_Districte", "District")
DF3 = DF3.withColumnRenamed("Any", "Year")

In [60]:
DF1.createOrReplaceTempView("taula1")
DF2.createOrReplaceTempView("taula2")
DF3.createOrReplaceTempView("taula3")

Comprovem

In [61]:
query = spark.sql("""
    SELECT DISTINCT(District)
    FROM taula3
""")
query.show()

+-------------------+
|           District|
+-------------------+
|          Les Corts|
|Sarria-Sant Gervasi|
|         Nou Barris|
|           Eixample|
|             Gracia|
|     Sants-Montjuic|
|        Sant Andreu|
|         Sant Marti|
|     Horta-Guinardo|
|       Ciutat Vella|
+-------------------+



In [68]:
DF = spark.sql("""
    SELECT t1.Year,t1.District, t1.Year_Price_month, t2.Year_Price_m2, t3.District_Income
    FROM taula1 t1, taula2 t2, taula3 t3
    WHERE t1.District = t2.District AND t1.District = t3.District and t1.Year = t2.Year and t1.Year = t3.Year
""")
DF.show()

+----+-------------------+------------------+------------------+------------------+
|Year|           District|  Year_Price_month|     Year_Price_m2|   District_Income|
+----+-------------------+------------------+------------------+------------------+
|2015|     Horta-Guinardo| 595.0114634146341| 9.773170731707317|32367.845528455284|
|2015|         Nou Barris|501.61857142857133| 8.373571428571429|27483.162393162394|
|2016|     Sants-Montjuic| 662.4172413793102|11.238620689655173| 31840.05982905983|
|2015|     Sants-Montjuic| 611.3019999999999|10.202333333333334|31367.478632478633|
|2019|     Sants-Montjuic| 823.6586206896551|12.972413793103447| 35258.96581196581|
|2020|         Nou Barris| 678.4699999999998|11.259999999999998|           30406.0|
|2016|             Gracia| 756.1959999999999|12.372000000000002| 37910.24137931035|
|2015|Sarria-Sant Gervasi|1083.3508333333332|13.027083333333332|60116.102040816324|
|2021|     Sants-Montjuic|  798.932142857143|12.678571428571427| 36198.78632

In [69]:
DF.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:exploited_zone/freshdata_exploited1.db") \
        .option("dbtable", "nova_base") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .mode("overwrite") \
        .save()

24/04/25 10:57:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 474037 ms exceeds timeout 120000 ms
24/04/25 10:57:57 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/25 10:57:58 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o